# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [52]:
import pandas as pd
import numpy as np
import re #regex

In [53]:
df = pd.read_csv('previsao_de_renda.csv')

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [55]:
import numpy as np
import statsmodels.api as sm
import patsy

In [56]:
#excluindo variáveis irrelevantes para o nosso propósito
df = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)


In [57]:
#Verificando valores ausentes
missing = df.isnull().sum()
missing

sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
dtype: int64

In [58]:
#imputação de mediana dos valores ausentes em tempo_emprego 
mediana_tempo_emprego = df['tempo_emprego'].median()

df['tempo_emprego'] = df['tempo_emprego'].fillna(mediana_tempo_emprego)

In [59]:
#verificando novamente se há valores nuloes
missing_values = df.isnull().sum()
missing_values

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64

In [60]:
#Verificando a contagem de entradas em cada coluna
df.count()

sexo                     15000
posse_de_veiculo         15000
posse_de_imovel          15000
qtd_filhos               15000
tipo_renda               15000
educacao                 15000
estado_civil             15000
tipo_residencia          15000
idade                    15000
tempo_emprego            15000
qt_pessoas_residencia    15000
renda                    15000
dtype: int64

**Agora que excluímos as colunas irrelevantes, imputamos a mediana na coluna 'tempo_emprego' e confirmamos que não há valores ausentes, daremos continuidade em nosso trabalho.**

**1 - Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.**

In [61]:
#transformação logartitmica para a  variável renda

df['log_renda'] = np.log(df['renda'])

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          15000 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
 12  log_renda              15000 non-null  float64
dtypes: bool(2), float64(4), int64(2), object(5)
memory usage: 1.3+ MB


**Agora temos duas variáveis relacionadas a renda: a 'renda' original e 'log_renda'**


In [63]:
#Criando novo DF sem a coluna 'renda' original
df2 = df.copy()
df2.drop(columns=['renda'])

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,log_renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8.994711
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,7.524102
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,7.720413
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,8.794942
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,8.775854
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,8.986019
14996,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,6.013699,1.0,9.219642
14997,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,6.404931
14998,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,8.117393


In [64]:
#Criando um modelo de RL sendo log_renda explicado com todas as variáveis 

formula = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

y, X = patsy.dmatrices(formula, data=df2)

model = sm.OLS(y, X)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Fri, 09 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:01:09   Log-Likelihood:                -16213.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14975   BIC:                         3.267e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.6950      0.224     29.902      0.000       6.256       7.134
sexo[T.M]                            0.7946      0.014     57.705      0.000       0.768       0.822
posse_de_veiculo[T.True]             0.0351      0.013      2.692      0.007       0.010       0.061
posse_de_imovel[T.True]              0.0898      0.013      6.984      0.000       0.065       0.115
tipo_renda[T.Bolsista]               0.2074      0.238      0.870      0.385      -0.260       0.675
tipo_renda[T.Empresário]             0.1543      0.015     10.459      0.000       0.125       0.183
tipo_renda[T.Pensionista]           -0.0862      0.021     -4.039      0.000      -0.128      -0.044
tipo_renda[T.Servidor público]       0.0572      0.022      2.604      0.009       0.014       0.100
educacao[T.Pós graduação]            0.0547      0.151      0.362      0.717      -0.241       0.351
educacao[T.Secundário]              -0.0720      0.056     -1.279      0.201      -0.182       0.038
educacao[T.Superior completo]        0.0477      0.057      0.841      0.400      -0.064       0.159
educacao[T.Superior incompleto]     -0.1024      0.063     -1.613      0.107      -0.227       0.022
estado_civil[T.Separado]             0.2754      0.107      2.568      0.010       0.065       0.486
estado_civil[T.Solteiro]             0.2291      0.105      2.178      0.029       0.023       0.435
estado_civil[T.União]               -0.0382      0.023     -1.661      0.097      -0.083       0.007
estado_civil[T.Viúvo]                0.2855      0.108      2.638      0.008       0.073       0.498
tipo_residencia[T.Casa]             -0.0314      0.052     -0.604      0.546      -0.133       0.071
tipo_residencia[T.Com os pais]      -0.0116      0.058     -0.199      0.843      -0.126       0.103
tipo_residencia[T.Comunitário]      -0.1108      0.103     -1.072      0.284      -0.313       0.092
tipo_residencia[T.Estúdio]           0.0618      0.094      0.658      0.510      -0.122       0.246
tipo_residencia[T.Governamental]    -0.0014      0.062     -0.023      0.981      -0.122       0.120
qtd_filhos                          -0.2364      0.104     -2.263      0.024      -0.441      -0.032
idade                                0.0052      0.001      6.798      0.000       0.004       0.007
tempo_emprego                        0.0618      0.001     60.433      0.000       0.060       0.064
qt_pessoas_residencia                0.2605      0.104      2.498      0.012       0.056       0.465
==============================================================================
Omnibus:                        0.717   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.699   Jarque-Bera (JB):                0.687
Skew:             

**Mantenha sempre a categoria mais frequente como casela de referência**

In [65]:
#identificando as variáveis categóricas de df2

variaveis_categoricas = df2.select_dtypes(include=['object']).columns.to_list()
variaveis_categoricas

['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']

In [66]:
#Identificando a categoria mais frequente para cada variável categórica

categorias = variaveis_categoricas

#Criando um dicionário com um filtro das categorias mais frequentes
categorias_mais_frequentes = {var : df2[var].value_counts().idxmax() for var in categorias}
categorias_mais_frequentes


{'sexo': 'F',
 'tipo_renda': 'Assalariado',
 'educacao': 'Secundário',
 'estado_civil': 'Casado',
 'tipo_residencia': 'Casa'}

**Ajustando o modelo de regressão linear com as categorias mais frequentes como referência**

In [67]:
#Log_renda explicado pelas variáveis categóricas mais frequentes.
#C - transforma a variável em dummy. Treatmen(" ") seleciona a categoria da variável como referência.
formula = 'log_renda ~ C(sexo, Treatment("F")) + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia, Treatment("Casa"))'

# Criando e ajustando o modelo de regressão linear
y, X = patsy.dmatrices(formula, data=df2)
modelo = sm.OLS(y, X)
results = modelo.fit()

# Exibindo o resumo do modelo ajustado
resumo = results.summary()
resumo


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     158.0
Date:                Fri, 09 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:01:09   Log-Likelihood:                -18121.
No. Observations:               15000   AIC:                         3.628e+04
Df Residuals:                   14981   BIC:                         3.642e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.9564      0.013    606.186      0.000       7.931       7.982
C(sexo, Treatment("F"))[T.M]                                    0.6972      0.015     47.608      0.000       0.669       0.726
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.4004      0.271      1.480      0.139      -0.130       0.931
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.0685      0.017      4.108      0.000       0.036       0.101
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.1520      0.019     -7.877      0.000      -0.190      -0.114
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.2168      0.025      8.757      0.000       0.168       0.265
C(educacao, Treatment("Secundário"))[T.Primário]                0.0252      0.064      0.394      0.693      -0.100       0.150
C(educacao, Treatment("Secundário"))[T.Pós graduação]          -0.0699      0.159     -0.439      0.661      -0.382       0.242
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.0970      0.014      6.796      0.000       0.069       0.125
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.1240      0.035     -3.538      0.000      -0.193      -0.055
C(estado_civil, Treatment("Casado"))[T.Separado]                0.0430      0.029      1.505      0.132      -0.013       0.099
C(estado_civil, Treatment("Casado"))[T.Solteiro]               -0.0376      0.021     -1.804      0.071      -0.078       0.003
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0695      0.026     -2.673      0.008      -0.120      -0.019
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.0460      0.032      1.417      0.156      -0.018       0.110
C(tipo_residencia, Treatment("Casa"))[T.Aluguel]               -0.1241      0.059     -2.108      0.035      -0.240      -0.009
C(tipo_residencia, Treatment("Casa"))[T.Com os pais]           -0.1613      0.032     -5.000      0.000      -0.225      -0.098
C(tipo_residencia, Treatment("Casa"))[T.Comunitário]           -0.2513      0.102     -2.465      0.014      -0.451      -0.051
C(tipo_residencia, Treatment("Casa"))[T.Estúdio]                0.1802      0.089      2.015      0.044       0.005       0.356
C(tipo_residencia, Treatment("Casa"))[T.Governamental]          0.0135      0.039      0.348      0.728      -0.063       0.090
==============================================================================
Omnibus:                      145.795   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              166.268
Skew:                         

**R-squared do modelo (original) 0.160**

**Interpretando os resultados:**

*Coeficiente Positivo: Se o coeficiente estimado para uma categoria específica for positivo, isso sugere que, mantendo todas as outras variáveis constantes, essa categoria tem uma influência positiva na log_renda. Em outras palavras, as pessoas nessa categoria tendem a ter uma log_renda estimada mais alta em comparação com a categoria de referência.

*Coeficiente Negativo: Se o coeficiente estimado for negativo, isso indica uma influência negativa na log_renda para essa categoria específica em comparação com a categoria de referência. As pessoas nessa categoria tendem a ter uma log_renda estimada mais baixa.

*Categoria de Referência: A categoria de referência em si também fornece informações. Os coeficientes para a categoria de referência são geralmente iguais a zero (ou muito próximos de zero), pois eles representam a linha de base para comparação. As outras categorias são comparadas à categoria de referência.

Portanto, ao analisar os coeficientes estimados para as variáveis categóricas, pode-se ter uma ideia de quais categorias tendem a ter uma renda mais alta ou mais baixa em relação à categoria de referência.

**2 - Remova a variável menos significante e analise:**


**Conforme observado acima, nosso maior p_valor encontra-se na variável C(tipo_residencia, Treatment("Casa"). No modelo abaixo iremos excluí-la e compararemos com o R-quadrado anterior (160).**

In [72]:
#Excluindo C(tipo_residencia, Treatment("Casa") do modelo

formula = 'log_renda ~ C(sexo, Treatment("F")) + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado"))'

# Criando e ajustando o modelo de regressão linear
y, X = patsy.dmatrices(formula, data=df2)
modelo = sm.OLS(y, X)
results = modelo.fit()

# Exibindo o resumo do modelo ajustado
resumo = results.summary()
resumo

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     215.3
Date:                Fri, 09 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:08:13   Log-Likelihood:                -18141.
No. Observations:               15000   AIC:                         3.631e+04
Df Residuals:                   14986   BIC:                         3.642e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.9488      0.013    613.056      0.000       7.923       7.974
C(sexo, Treatment("F"))[T.M]                                    0.6936      0.015     47.510      0.000       0.665       0.722
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.4110      0.271      1.517      0.129      -0.120       0.942
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.0729      0.017      4.377      0.000       0.040       0.106
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.1413      0.019     -7.344      0.000      -0.179      -0.104
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.2197      0.025      8.870      0.000       0.171       0.268
C(educacao, Treatment("Secundário"))[T.Primário]                0.0105      0.064      0.164      0.870      -0.115       0.136
C(educacao, Treatment("Secundário"))[T.Pós graduação]          -0.0620      0.160     -0.389      0.697      -0.375       0.251
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.0940      0.014      6.594      0.000       0.066       0.122
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.1381      0.035     -3.946      0.000      -0.207      -0.069
C(estado_civil, Treatment("Casado"))[T.Separado]                0.0410      0.029      1.431      0.152      -0.015       0.097
C(estado_civil, Treatment("Casado"))[T.Solteiro]               -0.0439      0.021     -2.110      0.035      -0.085      -0.003
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0725      0.026     -2.788      0.005      -0.123      -0.022
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.0462      0.033      1.421      0.155      -0.018       0.110
==============================================================================
Omnibus:                      153.271   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              175.992
Skew:                           0.197   Prob(JB):                     6.08e-39
Kurtosis:                       3.355   Cond. No.                         48.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Notamos que após a exclusão desta variável C(tipo_residencia, Treatment("Casa"), nosso R-quadrado diminuiu um pouco,notamos porém que agora a variável C(educacao, Treatment("Secundário"))[T.Primário] possui um p_valor bastante elevado de 0.870. Desta forma também iremos excluí-la do modelo e compararemos o resultado adiante.**

In [79]:
# Ajustando o modelo de regressão linear com as categorias mais frequentes e excluindo 
#.   C(tipo_residencia, Treatment("Casa") e  C(educacao, Treatment("Secundário")
formula = 'log_renda ~ C(sexo, Treatment("F")) + C(tipo_renda, Treatment("Assalariado")) + C(estado_civil, Treatment("Casado"))'

# Criando e ajustando o modelo de regressão linear
y, X = patsy.dmatrices(formula, data=df2)
modelo = sm.OLS(y, X)
results = modelo.fit()

# Exibindo o resumo do modelo ajustado
resumo = results.summary()
resumo

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     301.9
Date:                Fri, 09 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:30:17   Log-Likelihood:                -18175.
No. Observations:               15000   AIC:                         3.637e+04
Df Residuals:                   14990   BIC:                         3.645e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.9753      0.012    673.056      0.000       7.952       7.998
C(sexo, Treatment("F"))[T.M]                                    0.6906      0.015     47.219      0.000       0.662       0.719
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.4785      0.271      1.764      0.078      -0.053       1.010
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.0806      0.017      4.848      0.000       0.048       0.113
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.1437      0.019     -7.487      0.000      -0.181      -0.106
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.2373      0.025      9.606      0.000       0.189       0.286
C(estado_civil, Treatment("Casado"))[T.Separado]                0.0461      0.029      1.609      0.108      -0.010       0.102
C(estado_civil, Treatment("Casado"))[T.Solteiro]               -0.0435      0.021     -2.090      0.037      -0.084      -0.003
C(estado_civil, Treatment("Casado"))[T.União]                  -0.0770      0.026     -2.958      0.003      -0.128      -0.026
C(estado_civil, Treatment("Casado"))[T.Viúvo]                   0.0393      0.033      1.208      0.227      -0.024       0.103
==============================================================================
Omnibus:                      152.762   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              175.174
Skew:                           0.197   Prob(JB):                     9.15e-39
Kurtosis:                       3.353   Cond. No.                         45.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Com base no resultado acima, notamos que após a exclusão das variáveis anteriores, nosso R-quadrado deminuiu um pouco mais (153). Notamos que a variável C(estado_civil, Treatment("Casado") possui o maior p_value resultante. Agora também a excluiremos do nosso modelo e compararemos o resultado**

In [78]:
# Modelo sem as variáveis  C(tipo_residencia, Treatment("Casa"), C(educacao, Treatment("Secundário")
#.   e C(estado_civil, Treatment("Casado")
formula = 'log_renda ~ C(sexo, Treatment("F")) + C(tipo_renda, Treatment("Assalariado"))'

# Criando e ajustando o modelo de regressão linear
y, X = patsy.dmatrices(formula, data=df2)
modelo = sm.OLS(y, X)
results = modelo.fit()

# Exibindo o resumo do modelo ajustado
resumo = results.summary()
resumo

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     539.2
Date:                Fri, 09 Feb 2024   Prob (F-statistic):               0.00
Time:                        14:26:43   Log-Likelihood:                -18184.
No. Observations:               15000   AIC:                         3.638e+04
Df Residuals:                   14994   BIC:                         3.643e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.9695      0.011    733.110      0.000       7.948       7.991
C(sexo, Treatment("F"))[T.M]                                    0.6891      0.014     47.693      0.000       0.661       0.717
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.4843      0.271      1.785      0.074      -0.048       1.016
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.0772      0.017      4.651      0.000       0.045       0.110
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.1383      0.019     -7.330      0.000      -0.175      -0.101
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.2365      0.025      9.572      0.000       0.188       0.285
==============================================================================
Omnibus:                      153.064   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              176.323
Skew:                           0.196   Prob(JB):                     5.15e-39
Kurtosis:                       3.359   Cond. No.                         45.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Conclusão: iniciamos nosso modelo com todas as variáveis com as categorias mais frequentes no qual obtivemos um resultado R-quadrado de 0.160. Com isso em mente, iniciamos a exclusão das variáveis menos significativas estatísticamente levando em consideração p-valores maiores que 5%. Nosso modelo final resultou com um R-quadrado de 0.152 e apenas duas variáveis totalmente signitificativas estatisticamente com p-valores 0.000, sendo elas: C(sexo, Treatment("F") e C(estado_civil, Treatment("Casado").
Podemos concluir que embora nosso R-quadrado tenha diminuido um pouco (de 16% para 15%), um modelo mais simplificado pode ser vantajoso para o nosso objetivo.**